In [208]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [240]:
def getPageData(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    #FIND THE STORY TITLES ON ONE PAGE
    story_titles = soup.findAll('span', attrs={"class":"title"})
    titles = []
    for row in story_titles:
        titles.append(row.text.strip())

    #FIND THE STORY AUTHORS ON ONE PAGE
    story_authors = soup.findAll('span', attrs={"class":"byline"})
    authors = []
    for row in story_authors:
        authors.append(row.text.strip())
    for x in range(len(authors)):
        authors[x] = re.sub('by ','',authors[x])

    #GET STORY VOTES FOR ONE PAGE
    story_votes = soup.findAll('td', attrs={"class":"info"})
    votes = []
    for x in story_votes:
        votes.append(x.text)
    votes2 = votes
    for x in range(len(votes)):
        votes2[x] = re.sub(r'\n', '', str(votes[x]))
        votes2[x] = re.sub(r'\xa0boosters', '', str(votes2[x]))

    #GET STORY TAGS FOR ONE PAGE
    story_tags = soup.findAll('p', attrs={"class":"tags"})
    tags = []
    for x in story_tags:
        tags.append(x.text)
    for x in range(len(tags)):
        tags[x] = re.sub(' ', '-', str(tags[x]))
        tags[x] = re.sub(r'\n', ' ', str(tags[x]))
        tags[x] = re.split(' ', str(tags[x]))
    
    
    #GET TITLES FOR STORIES WITH MOST SHARED BOOSTERS
    links = []
    for link in soup.find_all('a', attrs={'href': re.compile("^https://topwebfiction.com/listings/")}):
        links.append(link.get('href'))

    def get_related_links_soup(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        related_stories_soup = soup.findAll('span', attrs={"class":"title"})
        return(related_stories_soup)

    def get_related_links(soup):
        link_list = []
        for x in range(len(soup)):
            link_list.append(soup[x].text)
        return(link_list[10:])

    def relatedLinkFetch(url): 
        linksoup = get_related_links_soup(url)
        return(get_related_links(linksoup))

    rel_stories = []
    for x in links:
        rel_stories.append(relatedLinkFetch(x))
    
    #REMOVE STORIES WITH NO TAGS. I KNOW THIS DOESNT SCALE BUT I DID MY BEST
    if ('Google' in titles) == True:
        ind = titles.index('Google')
        del(titles[ind])
        del(authors[ind])
        del(votes[ind])
        del(rel_stories[ind])
    
    if ('Out of New York City' in titles) == True:
        ind = titles.index('Out of New York City')
        del(titles[ind])
        del(authors[ind])
        del(votes[ind])
        del(rel_stories[ind])
    
    if ('Project Shelf Life' in titles) == True:
        ind = titles.index('Project Shelf Life')
        del(titles[ind])
        del(authors[ind])
        del(votes[ind])
        del(rel_stories[ind])  
    
    #PUT ALL THE COLUMNS TOGETHER INTO A DATAFRAME
    page_df = pd.DataFrame(
        {'Title': titles,
         'Author': authors,
         'Votes': votes,
         'Tags': tags,
         'Related Stories': rel_stories
        })
    
    return(page_df)

In [261]:
dfs = []
base_url = 'https://topwebfiction.com/?ranking=at&page='
for i in range(1,16):
    df = getPageData(base_url+str(i))
    dfs.append(df)
    
df = pd.concat(dfs)
df.index = range(len(df))

In [242]:
df.head()

,Title,Author,Votes,Tags,Related Stories
0,A Practical Guide to Evil,ErraticErrata,47605,"[adventure, anti-hero, coming-of-age, fantasy,...","[The Gods are Bastards, The Wandering Inn, Wor..."
1,Worm,Wildbow,39128,"[action, adventure, complete, crime, dark, hig...","[Pact, Twig, Ward, A Practical Guide to Evil, ..."
2,Ward,wildbow,24117,"[action, adventure, alternate-universe, crime,...","[Worm, Twig, Pact, A Practical Guide to Evil, ..."
3,The Wandering Inn,pirateaba,22326,"[fantasy, magic, ]","[A Practical Guide to Evil, The Gods are Basta..."
4,Metaworld Chronicles,Wutosama,20019,"[fantasy, magic, relationships, young-adult, ]","[A Practical Guide to Evil, The Wandering Inn,..."


In [256]:
tags_dum = pd.get_dummies(df['Tags'].apply(pd.Series).stack()).sum(level=0)
related_stories_dum = pd.get_dummies(df['Related Stories'].apply(pd.Series).stack()).sum(level=0)
tags_df = pd.concat([df, tags_dum], axis=1)
rs_df = pd.concat([df, related_stories_dum, axis=1])

In [264]:
tags_df = pd.concat([df, tags_dum], axis=1)
rs_df = pd.concat([df, related_stories_dum], axis=1)

In [278]:
tags_df = tags_df.drop(['Tags', 'Related Stories'], axis=1)
rs_df = rs_df.drop(['Tags', 'Related Stories'], axis=1)

KeyError: "['Tags' 'Related Stories'] not found in axis"

In [279]:
tags_df.head()

,Title,Author,Votes,,action,addiction,adventure,ai,airships,aliens,...,western,witches,wizards,wolves,wuxia,xianxia,youkai,young-adult,young-love,zombies
0,A Practical Guide to Evil,ErraticErrata,47605,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,Worm,Wildbow,39128,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ward,wildbow,24117,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,The Wandering Inn,pirateaba,22326,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Metaworld Chronicles,Wutosama,20019,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [275]:
rs_df.head()

,Title,Author,Votes,100 Luck and the Dragon Tamer Skill!,23 Pangbourne Place,A Bad Idea,A Devil in Gods Country,A Grey World,A Journey of Black and Red,A Lament of Gods and Monsters,...,World Domination in Retrospect,World-ruling Dungeon,Worm,Worth the Candle,"Yes, Your Highness",Your Typical Isekai LitRPG,Zeppelins are What Dreams are Made of,aka,asa kraiya,blacklight
0,A Practical Guide to Evil,ErraticErrata,47605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Worm,Wildbow,39128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Ward,wildbow,24117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,The Wandering Inn,pirateaba,22326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Metaworld Chronicles,Wutosama,20019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [322]:
test = []
test.append(int(tags_df.iloc[0,2]))

In [333]:
for y in range(2, len(tags_df.columns)):
    for x in range(len(tags_df.iloc[:,y])):
        try:
            tags_df.iloc[x,y] = int(tags_df.iloc[x,y])
        except:
            tags_df.iloc[x,y] = 0

In [337]:
type(tags_df.iloc[0,2])

int

In [306]:
sum(int(tags_df.iloc[:,2]))

TypeError: cannot convert the series to <class 'int'>

In [339]:
col_sums = []
for x in range(2, len(tags_df.columns)):
    col_sums.append(sum(tags_df.iloc[:,x]))

In [344]:
type(tags_df.columns)

pandas.core.indexes.base.Index